In [1]:
#Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore')

In [2]:

# Read the csv files
input = pd.read_csv("Train.csv")

#print all columns to understand the dataset
input.head()

,ID,CONSOLE,YEAR,CATEGORY,PUBLISHER,RATING,CRITICS_POINTS,USER_POINTS,SalesInMillions
0,2860,ds,2008,role-playing,Nintendo,E,2.833333,0.303704,1.779257
1,731,wii,2012,simulation,Konami Digital Entertainment,E10+,13.200000,1.640000,0.215050
2,495,pc,2019,shooter,Activision,M,4.562500,0.006410,0.534402
3,2641,ps2,2002,sports,Electronic Arts,E,4.181818,0.326923,1.383964
4,811,ps3,2013,action,Activision,M,2.259259,0.032579,0.082671


In [4]:
input.isnull().sum()

ID                 0
CONSOLE            0
YEAR               0
CATEGORY           0
PUBLISHER          0
RATING             0
CRITICS_POINTS     0
USER_POINTS        0
SalesInMillions    0
dtype: int64

In [5]:
input = input.drop(columns=['ID'])
train, test = train_test_split(input, test_size=0.2, random_state=42, shuffle=True)
     

In [9]:
from pandas_profiling import ProfileReport
report = ProfileReport(train, title="Report", html={'style': {'full_width':True}}, explorative=True, missing_diagrams={'bar': True})

In [10]:
report.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
#Save the report in file
report.to_file("pandas_profiling_report.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
#Sales of games that happened corresponding to each console. 
df = pd.DataFrame(train.groupby(['CONSOLE']).agg({'SalesInMillions': 'sum'}))
     

df.plot.bar(figsize=(12, 6))

<Axes: xlabel='CONSOLE'>

In [13]:

df = pd.DataFrame(train.groupby(['YEAR']).agg({'SalesInMillions': 'sum'}))
     

df.plot.bar(figsize=(12, 6))

<Axes: xlabel='YEAR'>

In [14]:

df = pd.DataFrame(train.groupby(['CATEGORY']).agg({'SalesInMillions': 'sum'}))
     

df.plot.bar(figsize=(12, 6))

<Axes: xlabel='CATEGORY'>

In [15]:


import catboost as cat
cat_feat = ['CONSOLE','CATEGORY', 'PUBLISHER', 'RATING']
features = list(set(train.columns)-set(['SalesInMillions']))
target = 'SalesInMillions'
model = cat.CatBoostRegressor(random_state=100,cat_features=cat_feat,verbose=0)
model.fit(train[features],train[target])
     

In [16]:
y_true= pd.DataFrame(data=test[target], columns=['SalesInMillions'])
test_temp = test.drop(columns=[target])
     

y_pred = model.predict(test_temp[features])
     

from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_true, y_pred))
print(rmse)

1.5331132299453223


In [17]:
import pickle
filename = 'finalized_model.sav'
     

pickle.dump(model, open(filename, 'wb'))
     

loaded_model = pickle.load(open(filename, 'rb'))
     

test_temp[features].head(1)

,CONSOLE,CRITICS_POINTS,YEAR,RATING,PUBLISHER,CATEGORY,USER_POINTS
3272,ps3,2.806452,2015,M,Take-Two Interactive,shooter,0.009848


In [18]:

loaded_model.predict(test_temp[features].head(1))

array([3.44258967])